# Interpret Models

You can use Azure Machine Learning to interpret a model by using an *explainer* that quantifies the amount of influence each feature contribues to the predicted label. There are many common explainers, each suitable for different kinds of modeling algorithm; but the basic approach to using them is the same.

## Install SDK packages

Let's start by ensuring that you have the latest version of the Azure ML SDK installed, including the *explain* optional package. In addition, you'll install the Azure ML Interpretability library. You can use this to interpret many typical kinds of model, even if they haven't been trained in an Azure ML experiment or registered in an Azure ML workspace.

In [1]:
!pip install --upgrade --user azureml-sdk[notebooks,explain]
!pip install --upgrade --user azureml-interpret

  Using cached azureml_sdk-1.49.0-py3-none-any.whl (2.7 kB)
  Using cached azureml_train_core-1.49.0.post1-py3-none-any.whl (8.6 MB)
  Using cached azureml_pipeline-1.49.0-py3-none-any.whl (2.4 kB)
  Using cached azureml_dataset_runtime-1.49.0-py3-none-any.whl (2.3 kB)
  Using cached azureml_train_automl_client-1.49.0.post1-py3-none-any.whl (136 kB)
  Using cached azureml_explain_model-1.49.0-py3-none-any.whl (20 kB)
  Using cached azureml_widgets-1.49.0-py3-none-any.whl (14.1 MB)
  Using cached azureml_contrib_notebook-1.49.0-py3-none-any.whl (28 kB)
  Using cached azureml_pipeline_core-1.49.0-py3-none-any.whl (312 kB)
  Using cached nbconvert-5.6.1-py2.py3-none-any.whl (455 kB)
  Using cached jupyter_client-6.1.12-py3-none-any.whl (112 kB)
  Using cached papermill-1.2.1-py2.py3-none-any.whl (31 kB)
  Using cached azureml_dataprep-4.9.6-py3-none-any.whl (38.4 MB)
  Using cached azureml_pipeline_steps-1.49.0-py3-none-any.whl (69 kB)
  Using cached azureml_automl_core-1.49.1.post2-py3-n

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
werkzeug 2.2.2 requires MarkupSafe>=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
spyder-kernels 2.4.1 requires jup

In [4]:
!pip install --user --upgrade numpy

  Using cached numpy-1.24.2-cp310-cp310-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.2
    Uninstalling numpy-1.22.2:
      Successfully uninstalled numpy-1.22.2


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\admin\\AppData\\Local\\Temp\\pip-uninstall-uitvshl0\\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll'
Check the permissions.



## Explain a model

Let's start with a model that is trained outside of Azure Machine Learning - Run the cell below to train a decision tree classification model.

In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

# Separate features and labels
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

C:\Users\admin\AppData\Roaming\Python\Python310\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\admin\AppData\Roaming\Python\Python310\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\admin\AppData\Roaming\Python\Python310\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Loading Data...
Training a decision tree model
Accuracy: 0.89
AUC: 0.8763719588039204
Model trained.


The training process generated some model evaluation metrics based on a hold-back validation dataset, so you have an idea of how accurately it predicts; but how do the features in the data influence the prediction?

### Get an explainer for the model

Let's get a suitable explainer for the model from the Azure ML interpretability library you installed earlier. There are many kinds of explainer. In this example you'll use a *Tabular Explainer*, which is a "black box" explainer that can be used to explain many kinds of model by invoking an appropriate [SHAP](https://github.com/slundberg/shap) model explainer.

In [5]:
from interpret.ext.blackbox import TabularExplainer

# "features" and "classes" fields are optional
tab_explainer = TabularExplainer(model,
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

C extension was not built during install!


The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


TabularExplainer ready!


### Get *global* feature importance

The first thing to do is try to explain the model by evaluating the overall *feature importance* - in other words, quantifying the extent to which each feature influences the prediction based on the whole training dataset.

In [6]:
# you can use the training data or the test data here
global_tab_explanation = tab_explainer.explain_global(X_train)

# Get the top features by importance
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

  0%|          | 0/7000 [00:00<?, ?it/s]

Pregnancies : 0.22859005971817323
BMI : 0.1845300849368329
SerumInsulin : 0.12118250476190488
Age : 0.12107678741496777
PlasmaGlucose : 0.11261695330417862
TricepsThickness : 0.04007569470359593
DiastolicBloodPressure : 0.032914588386783176
DiabetesPedigree : 0.025947302575315745


The feature importance is ranked, with the most important feature listed first.

### Get *local* feature importance

So you have an overall view, but what about explaining individual observations? Let's generate *local* explanations for individual predictions, quantifying the extent to which each feature influenced the decision to predict each of the possible label values. In this case, it's a binary model, so there are two possible labels (non-diabetic and diabetic); and you can quantify the influence of each feature for each of these label values for individual observations in a dataset. You'll just evaluate the first two cases in the test dataset.

In [7]:
# Get the observations we want to explain (the first two)
X_explain = X_test[0:2]

# Get predictions
predictions = model.predict(X_explain)

# Get local explanations
local_tab_explanation = tab_explainer.explain_local(X_explain)

# Get feature names and importance for each possible label
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

  0%|          | 0/2 [00:00<?, ?it/s]

Support for not-diabetic
	Observation 1
		 BMI : 0.34551190476190474
		 SerumInsulin : 0.326178571428571
		 TricepsThickness : 0.14772619047619068
		 Age : 0.0
		 DiabetesPedigree : 0.0
		 DiastolicBloodPressure : 0.0
		 Pregnancies : 0.0
		 PlasmaGlucose : -0.13984523809523786
		 ----------
		 Total: 0.6795714285714286 Prediction: not-diabetic
	Observation 2
		 BMI : 0.49338095238095175
		 Pregnancies : 0.4491738095238091
		 DiastolicBloodPressure : 0.07351190476190483
		 TricepsThickness : 0.00895000000000018
		 DiabetesPedigree : -0.0020309523809522445
		 Age : -0.09169285714285669
		 PlasmaGlucose : -0.10446428571428557
		 SerumInsulin : -0.14725714285714264
		 ----------
		 Total: 0.6795714285714288 Prediction: not-diabetic
Support for diabetic
	Observation 1
		 PlasmaGlucose : 0.13984523809523786
		 Age : 0.0
		 DiabetesPedigree : 0.0
		 DiastolicBloodPressure : 0.0
		 Pregnancies : 0.0
		 TricepsThickness : -0.14772619047619068
		 SerumInsulin : -0.326178571428571
		 BMI : -0.34

## Adding explainability to a model training experiment

As you've seen, you can generate explanations for models trained outside of Azure Machine Learning; but when you use experiments to train and register models in your Azure Machine Learning workspace, you can generate model explanations and log them.

Run the code in the following cell to connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [9]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.get(name = "aml-ws2",
                      subscription_id="84a5808b-5549-459a-98f2-f102e84fa1bb",
                      resource_group="EY23")
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.49.0 to work with aml-ws2


### Train and explain a model using an experiment

OK, let's create an experiment and put the files it needs in a local folder - in this case we'll just use the same CSV file of diabetes data to train the model.

In [11]:
import os, shutil
from azureml.core import Experiment

# Create a folder for the experiment files
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

'diabetes_train_and_explain\\diabetes.csv'

Now we'll create a training script that looks similar to any other Azure ML training script except that is includes the following features:

- The same libraries to generate model explanations we used before are imported and used to generate a global explanation
- The **ExplanationClient** library is used to upload the explanation to the experiment output

In [12]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Import Azure ML run library
from azureml.core.run import Run

# Import libraries for model explanation
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# Separate features and labels
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# Get explanation
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Get an Explanation Client and upload the explanation
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# Complete the run
run.complete()

Writing diabetes_train_and_explain/diabetes_training.py


Now you can run the experiment. Note that the **azureml-interpret** library is included in the training environment so the script can create a **TabularExplainer** and use the **ExplainerClient** class.

In [13]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails


# Create a Python environment for the experiment
explain_env = Environment("explain-env")

# Create a set of package dependencies (including the azureml-interpret package)
packages = CondaDependencies.create(conda_packages=['scikit-learn','pandas','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret'])
explain_env.python.conda_dependencies = packages

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                      script='diabetes_training.py',
                      environment=explain_env) 

# submit the experiment
experiment_name = 'diabetes_train_and_explain'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes_train_and_explain_1681113677_c4dade97',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2023-04-10T08:18:37.131877Z',
 'endTimeUtc': '2023-04-10T08:19:36.872193Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '84b164ea-0f64-49a2-b8c1-efc8bf2baa7b'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'explain-env',
   'version': 'Autosave_2023-04-10T08:01:19Z_f987c458',
   'assetId': 'azureml://locations/westus/workspace

## Retrieve the feature importance values

With the experiment run completed, you can use the **ExplanationClient** class to retrieve the feature importance from the explanation registered for the run.

In [14]:
from azureml.interpret import ExplanationClient

# Get the feature explanations
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

Feature	Importance
Pregnancies 	 0.22241669017773605
Age 	 0.10397090119692536
BMI 	 0.09640834466086473
SerumInsulin 	 0.06914593012498527
PlasmaGlucose 	 0.049602730998427935
TricepsThickness 	 0.022150138661136572
DiastolicBloodPressure 	 0.018597062215891913
DiabetesPedigree 	 0.01363786418670087


## View the model explanation in Azure Machine Learning studio

You can also click the **View run details** link in the Run Details widget to see the run in Azure Machine Learning studio, and view the **Explanations** tab. Then:

1. Select the **Tabular Explanation** explainer.
2. View the **Global Importance** chart, which shows the overall global feature importance.
3. View the **Summary Importance** chart, which shows each data point from the test data in a *swarm*, *violin*, or *box* plot.
4. Select an individual point to see the **Local Feature Importance** for the individual prediction for the selected data point.


**More Information**: For more information about using explainers in Azure ML, see [the documentation](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability). 